In [2]:

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [3]:
import torch
import pandas as pd
from src.tools.preprocess import preprocess_data
import matplotlib.pyplot as plt
import rasterio
from skimage import exposure
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import joblib

from sklearn.metrics import classification_report

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
data = pd.read_csv('../data/processed/data.csv')
data_clean, anomalies = preprocess_data(data[data["dataset"] == "train"])
data_clean.head()

/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3943.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_clean.loc[non_null_mask & outlier_mask, col] = clean_mean
/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1301674.4039252088' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_clean.loc[non_null_mask & outlier_mask, col] = clean_mean
/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future ve

,Unnamed: 0,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,...,ndwi,gndvi,savi,msavi,SDate_year,SDate_month,SDate_day,HDate_year,HDate_month,HDate_day
0,0.0,1326576.0,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25,2024-04-14,97.0,...,8.488031,0.127153,0.151125,4232.596191,2023,11,25,2024,4,14
1,1.0,1326577.0,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13,2024-04-26,82.0,...,11.493762,0.187815,0.282110,3249.392822,2023,11,13,2024,4,26
2,2.0,1326578.0,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19,2024-04-28,92.0,...,10.166884,0.206553,0.309869,3741.956055,2023,12,19,2024,4,28
3,3.0,1331836.0,Diseased,Paddy,Telangana,Medak,Kulcharam,2023-02-11,2024-11-04,91.0,...,10.446196,0.220995,0.309350,3673.396729,2023,2,11,2024,11,4
4,4.0,1326579.0,Diseased,Paddy,Telangana,Medak,Kulcharam,2023-12-12,2024-05-19,94.0,...,8.665929,0.160657,0.269563,4227.338379,2023,12,12,2024,5,19


In [7]:
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
from PIL import Image


def scale_band(band):
    """Scale band data to 0-255."""
    band_min = np.min(band)
    band_max = np.max(band)
    return ((band - band_min) / (band_max - band_min) * 255).astype(np.uint8)


class TiffDataset(Dataset):
    def __init__(self, directory, data, type="train", transform=None, gamma=0.6, contrast_stretch=True):
        self.directory = directory
        self.transform = transform
        self.gamma = gamma
        self.contrast_stretch = contrast_stretch
        self.images = data.loc[(data["tif_path"].str.endswith('.tif')) & (data["dataset"] == type),
                               "tif_path"
                               ].tolist()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.directory, self.images[idx])

        # Open the image file
        with rasterio.open(img_path) as src:
            # Read RGB bands (assuming Sentinel-2 band order: Red=B4, Green=B3, Blue=B2)
            red = src.read(3)
            green = src.read(2)
            blue = src.read(1)
            # Stack the scaled RGB bands into a single image
            rgb = np.dstack((red, green,
                            blue)).astype(np.uint8)

            # Convert NumPy array to PIL Image
            rgb = Image.fromarray(rgb)

            if self.transform:
                rgb = self.transform(rgb)

        return rgb

In [8]:
transform = transforms.Compose([
    transforms.Resize((16, 16)),
    transforms.RandomRotation(degrees=30),
    transforms.RandomAffine(translate=(0.1, 0.1), degrees=20),
    transforms.ToTensor()
])

In [9]:

train_dataset = TiffDataset(
    directory="../data", data=data_clean, type="train", transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TiffDataset(
    directory="../data", data=data_clean, type="test", transform=transform)
test_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [10]:
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, 3, stride=2, padding=1),  # 32x32 -> 16x16
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
            torch.nn.Dropout(0.2),

            torch.nn.Conv2d(64, 128, 3, stride=2, padding=1),  # 16x16 -> 8x8
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(128),
            torch.nn.Dropout(0.2),

            torch.nn.Conv2d(128, 256, 3, stride=2, padding=1),  # 8x8 -> 4x4
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(256),
            torch.nn.Flatten(),

            torch.nn.Linear(256 * 2 * 2, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),

            torch.nn.Linear(64, 5)  # Code latent réduit à 4
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(5, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),

            torch.nn.Linear(64, 256 * 2 * 2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(256 * 2 * 2),

            torch.nn.Unflatten(1, (256, 2, 2)),
            torch.nn.ConvTranspose2d(
                256, 128, 3, stride=2, padding=1, output_padding=1),  # 4x4 -> 8x8
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(128),

            torch.nn.ConvTranspose2d(
                128, 64, 3, stride=2, padding=1, output_padding=1),   # 8x8 -> 16x16
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),

            torch.nn.ConvTranspose2d(
                64, 3, 3, stride=2, padding=1, output_padding=1),    # 16x16 -> 32x32
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [11]:
autoencoder = Autoencoder().to(device)

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
